In [1]:
%load_ext autoreload

In [2]:
%autoreload
import pandas as pd
import numpy as np
from Analyzers import EdgeDataAnalyzers
eda = EdgeDataAnalyzers.EdgeDataAnalyzer()
ca = EdgeDataAnalyzers.CentralityAnalyzer()
gv = EdgeDataAnalyzers.GraphVisualizer()
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
from Analyzers import MetaDataAnalyzer
mda = MetaDataAnalyzer.MetaDataAnalyzer()
import networkx as nx
import os

FileNotFoundError: File b'C:\\Users\\User\\Documents\\citations-network-analysis\\Analyzers\\..\\Data\\citations_to_names.csv' does not exist

In [ ]:
meta_df = pd.read_csv('Data/transformed_metadata.csv')

In [ ]:
meta_df.head()

global stats - mean, sd: cases per topic (top 20 and others), word count, citations received, citations given, citations received per word, citations given per word, num_cases

In [ ]:
cols_to_tabulate=['judgment_word_count',
                    'num_cited_cases',
                    'num_citing_cases',
                    'cites_per_000_word',
                 ]
global_stats = np.round(mda.tabulate_global_stats(meta_df,
    cols_to_tabulate=cols_to_tabulate), 4)

global_stats.columns = ['All Cases']
global_stats.loc['_size'] = meta_df.shape[0]
global_stats.index = pd.Series(global_stats.index).apply(eda.readify_varnames)

Combine with stats by bench

In [ ]:
bench_stats = meta_df.groupby('bench').agg({'judgment_word_count': [np.mean,np.std],
                         'num_cited_cases': [np.mean, np.std],
                         'num_citing_cases': [np.mean, np.std],
                         'cites_per_000_word': [np.mean, np.std, np.size]
}).apply(np.round, axis=1, args=(4,)).transpose()[['yong','chan','menon']]
for x_i in bench_stats.index:
    if x_i[-1] == 'std':
        bench_stats.loc[x_i] = bench_stats.loc[x_i].apply(lambda x: '('+str(np.round(x,4))+')')
bench_stats = bench_stats.reset_index()
bench_stats.index = bench_stats['level_0'] + '_' +bench_stats['level_1']
bench_stats.index.name = None
bench_stats.index = pd.Series(bench_stats.index).apply(eda.readify_varnames)
bench_stats = bench_stats.drop(['level_0', 'level_1'], axis=1)
bench_stats.columns = [x.title()+' Bench' for x in bench_stats.columns]
bench_stats['Overall'] = global_stats
bench_stats.loc['Years In Sample'] = [6,6,6,18]
bench_stats.to_latex('Tables/bench_stats.tex')
bench_stats

In [ ]:
yearly_stats = np.round(eda.tabulate_time_series(meta_df, 
                                        ['judgment_word_count','num_cited_cases', 'num_citing_cases', 'cites_per_000_word',],
                                        'decision_year'), 4)
yearly_stats = eda.format_groupby_for_pub(yearly_stats)
yearly_stats.columns.name = 'Year Decided'
yearly_stats.to_latex('Tables/yearly_stats.tex')
yearly_stats

In [ ]:
def show_values_on_bars(axs):
    def _show_on_single_plot(ax):        
        for p in ax.patches:
            _x = p.get_x() + p.get_width() + 7
            _y = p.get_y()
            value = '{:.0f}'.format(p.get_width())
            ax.text(_x, _y, value, ha="center") 

    if isinstance(axs, np.ndarray):
        for idx, ax in np.ndenumerate(axs):
            _show_on_single_plot(ax)
    else:
        _show_on_single_plot(axs)

topic_cols = [col for col in meta_df.columns if col.startswith('has_topic')]
topic_counts = meta_df[topic_cols].sum().sort_values(ascending=True)
topic_counts.index = [x.replace('has_topic','').replace('_', ' ').title().strip() for x in topic_counts.index]
plt.figure(figsize=(10,12))
ax = topic_counts.plot(kind='barh', color='skyblue')

### stuff to dejunk the plot
show_values_on_bars(ax)
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.tick_params(top='off', bottom='off', left='off', right='off', labelleft='on', labelbottom='off')

plt.tight_layout()
plt.savefig('Images/charts/global_topic_counts.png')

In [ ]:
most_freq_topics = pd.Series(topic_counts.sort_values(ascending=False).index[0:10].values)
most_freq_topics_colnames = most_freq_topics.apply(lambda x: ('has_topic_'+x.lower()).strip())
topic_colnames = ['has_topic_civil procedure','has_topic_contract','has_topic_criminal procedure and sentencing',
                  'has_topic_tort', 'has_topic_criminal law', 'has_topic_evidence', 'has_topic_companies',
                  'has_topic_words and phrases', 'has_topic_words and phrases', 'has_topic_courts and jurisdiction'
                 ]
mda.tabulate_topical_stats(meta_df, cols_to_tabulate, topic_colnames).transpose()

In [ ]:
fig = plt.figure(figsize=(14,4))
plt.subplot(131)
plt.scatter(meta_df['judgment_word_count'] /1000, meta_df['num_citing_cases'])
plt.xlabel('Judgment Word Count (\'000s)')
plt.ylabel('Inward Citations')
plt.title('(1)')
plt.subplot(132)
plt.scatter(meta_df['judgment_word_count'] /1000, meta_df['num_cited_cases'])
plt.ylabel('Outward Citations')
plt.xlabel('Judgment Word Count (\'000s)')
plt.title('(2)')
plt.subplot(133)
plt.scatter(meta_df['num_citing_cases'], meta_df['num_cited_cases'])
plt.xlabel('Inward Citations')
plt.ylabel('Outward Citations')
plt.title('(3)')
plt.subplots_adjust(wspace=0.30)
plt.savefig('Images/cite_scatters.png')

In [ ]:
meta_df[cols_to_tabulate].corr()

In [ ]:
table = pd.DataFrame()
for topic in topic_colnames:
    topic_data =  meta_df.groupby(topic)[cols_to_tabulate].agg([np.mean, np.std, np.size])
    has_topic_data = topic_data.loc[1]
    table[topic] = has_topic_data

topic_stats = eda.format_groupby_for_pub(np.round(table,4)).transpose()
#drop extra obs columns as we apply np.count every time
topic_stats.columns = ['drop' if x == 'Observations In Sample' else x for x in topic_stats.columns[:-1]] + ['Observations In Sample']
topic_stats = topic_stats.drop('drop', axis=1)

#all this work to properly place std below the respective columns
topic_stats.columns = ['std_{}'.format(topic_stats.columns[i-1]) if x == '' else x for i, x in enumerate(topic_stats.columns)]
sds = topic_stats[[col for col in topic_stats if col.startswith('std')]]
topic_stats = topic_stats.drop([col for col in topic_stats if col.startswith('std')], axis=1)
for topic in topic_colnames:
    for stat in topic_stats:
        if stat.lower().startswith('obs'):
            continue
        topic_stats.loc[topic+'_std', stat] = sds.loc[topic, 'std_'+stat]

# need to sort them back in place so they appear correctly
topic_stats = topic_stats.sort_index().fillna(method='ffill')
topic_stats['colFromIndex'] = topic_stats.index
topic_stats = topic_stats.sort_values(['Observations In Sample', 'colFromIndex'], ascending=True).drop('colFromIndex', axis=1)

# finally, std are right...
topic_stats['Observations In Sample'] = topic_stats['Observations In Sample'].astype(int)
topic_stats['Observations In Sample'] = ['' if (i+1)%2==0 else val for i, val in enumerate(topic_stats['Observations In Sample'])]
topic_stats.index = [eda.readify_varnames(x) for x in topic_stats.index]
topic_stats.to_latex('Tables/topic_stats.tex')
topic_stats

In [ ]:
meta_df.groupby('has_topic_tort')[cols_to_tabulate].agg([np.mean, np.std, np.size])

In [ ]:
mda.get_perc_with_sole_topic(meta_df, 'has_topic_contract')

time series plot of word count, num_cases, num_cited, cites_per_word, mean_cited_ages

note that graph is not used in the paper

In [ ]:
def standardize(pd_srs):
    return (pd_srs - pd_srs.mean()) / pd_srs.std()

yearly = meta_df.groupby('decision_year')[['judgment_word_count','num_cited_cases', 'num_citing_cases', 'cites_per_000_word',]].agg(np.mean)
fig, ax = plt.subplots(figsize=(11,7))
ax1 = yearly['judgment_word_count'].plot(ax=ax, label='Judgment Word Count (left axis)')
ax1.set_ylabel('Word Count')
ax2 = yearly['num_cited_cases'].plot(ax=ax, secondary_y=True, linestyle='--')
ax2 = yearly['num_citing_cases'].plot(ax=ax, secondary_y=True, linestyle='-.')
ax2 = yearly['cites_per_000_word'].plot(ax=ax, secondary_y=True, linestyle=':')
plt.legend((ax1.lines[0],ax2.lines[0], ax2.lines[1], ax2.lines[2]), ('Judgment Word Count (left axis)',
                                                      'No. Outgoing Citations (right axis)',
                                                      'No. Incoming Citations (right axis)',
                                                      'Outgoing Citations Per Thousand Words (right axis)',))
ax1.set_xlabel('Year Decided')
ax2.set_ylabel('No. Citations')
# this is simply to make the years display as ints
from matplotlib.ticker import MaxNLocator
ax1.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout()
plt.savefig('Images/charts/wordcount_time_series.png')
plt.show()

To produce lagged correlations presented in the paper

In [ ]:
for i in range(0,4):
    print(yearly['num_citing_cases'].corr(
        yearly['judgment_word_count'].shift(-i)))

heatmap of case load presented as Figure 1

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,8))
hmap1, ax1 = mda.plot_yearmonth_map(ax1, meta_df, 'judgment_word_count', with_rowmeans=True, with_colmeans=True, cbar_kws={'shrink':0.5})
hmap2, ax2 = mda.plot_yearmonth_map(ax2, meta_df, 'case_title', np.size, with_rowmeans=True, with_colmeans=True, cbar_kws={'shrink':0.5})
ax2.set_ylabel('')
ax1.set_title('Average Judgment Word Counts', y = 1.125)
ax2.set_title('Average Number of Cases', y = 1.125)
plt.tight_layout()
plt.savefig('Images/charts/num_cases_word_heat.png')

In [ ]:
meta_df[(meta_df['has_topic_civil procedure'] == 1) | (meta_df['has_topic_criminal procedure and sentencing']==1)][[col for col in meta_df.columns if col.startswith('has_topic')]].sum(axis=1).value_counts()

In [ ]:
mda.get_stats_by_topic_num(meta_df, 'evidence', 1)

Overall graph plots. Only the directed graph global plot is used

In [ ]:
g = gv.prepare_graph(edge_df, 'citing_id', 'cited_id', create_using=nx.Graph())
g, importance = gv.prepare_plot(g, importance_thresh=3)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax = gv.plot_graph(ax, g, importance, size_multiplier=5, with_labels=True, edge_kwargs={'width':0.5,'alpha':0.25})

In [ ]:
imp_thresh = 1
dg = gv.prepare_graph(edge_df, 'citing_id', 'cited_id', create_using=nx.DiGraph())
sub_dg, sub_importance = gv.prepare_plot(dg, importance_thresh=imp_thresh)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12))
gv.plot_graph(ax, sub_dg, sub_importance, size_multiplier=25, edge_kwargs={'width':0.5,'alpha':0.25})
plt.tight_layout()
plt.savefig('Images/graphs/overall_directed_imp_thresh_{}.png'.format(imp_thresh))

Central cases - overall

In [ ]:
all_cent = ca.get_centrality_df(dg)

In [ ]:
all_rank_scores = ca.get_rank_scores(all_cent, meta_df, 'authority_score').head(10)

In [ ]:
all_rank_scores.case_title.tolist()

In [ ]:
format_rank_scores(all_rank_scores.head(5)).to_latex('Tables/global_rankings.tex')

In [ ]:
citing_spandeck_outdeg = eda.get_scores_for_cases_citing_case(775, 'hub_score', edge_df, all_cent)
citing_zurich_outdeg = eda.get_scores_for_cases_citing_case(987, 'hub_score', edge_df, all_cent)

Find the 9 cases citing Zurich that aren't tagged contract

In [ ]:
eda.find_cases_citing(987, edge_df[edge_df['citing_has_topic_contract']==0])

Find the top cases by each measure

In [ ]:
all_central_cases = eda.convert_id_df_to_case_names(ca.get_top_k_per_measure(all_cent, k = 1, cols_to_exclude=[]), True)
all_central_cases = all_central_cases[['in_degree']+ca.get_measures()]
all_central_cases = all_central_cases.drop(['in_degree_cent', 'out_degree_cent'], axis=1)
all_central_cases.columns = np.vectorize(eda.readify_varnames)(all_central_cases.columns.values)
all_central_cases = all_central_cases.rename({0:'Top Case'},axis=0).transpose()
all_central_cases.to_latex('Tables/global_top_cases.tex')
all_central_cases

## Time Series Analysis

In [ ]:
%autoreload
import pandas as pd
import numpy as np
from Analyzers import EdgeDataAnalyzers
eda = EdgeDataAnalyzers.EdgeDataAnalyzer()
ca = EdgeDataAnalyzers.CentralityAnalyzer()
gv = EdgeDataAnalyzers.GraphVisualizer()
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline
from Analyzers import MetaDataAnalyzer
mda = MetaDataAnalyzer.MetaDataAnalyzer()
import networkx as nx

In [ ]:
top_5_yrly = ca.get_top_k_over_time(edge_df, 'authority_score', '20000101', '20171231', 'Y')

In [ ]:
top_5_yrly.columns = [str(col) for col in top_5_yrly.columns]
top_5_yrly#.applymap(eda.get_case_name_from_id_num) uncomment to get names instead

In [ ]:
def get_relevant_subset(edge_df, topic_name):
    if not topic_name:
        return edge_df.copy()
    else:
        return edge_df[edge_df['citing_has_topic_'+topic_name]==1].copy()

def get_key_cases_ts(edge_df, rank_scores, key_cases, topic_name=''):
    relevant_subset = get_relevant_subset(edge_df, topic_name)
    case_time_series = {}
    for case in key_cases:
        case_name = eda.get_case_name_from_id_num(case)
        print('Getting scores for case {}'.format(case_name))
        start, end = '20000101', '20171231'
        case_measures_over_time = ca.get_all_case_measures_over_time(
            relevant_subset, case, '20000101', '20171231', 
            measures_to_include = [
                'in_degree_cent',
                'hub_score',
                'authority_score',
            ]
        )
        case_time_series[case] = case_measures_over_time
    return case_time_series
    
def get_topic_rank_scores(edge_df, meta_df, topic_name=''):
    relevant_subset = get_relevant_subset(edge_df, topic_name)    
    topic_dg = gv.prepare_graph(relevant_subset, 'citing_id', 'cited_id', create_using=nx.DiGraph())
    topic_central_df = ca.get_centrality_df(topic_dg)
    rank_scores = ca.get_rank_scores(topic_central_df, meta_df, 'authority_score')
    return rank_scores    

In [ ]:
temp = pd.read_csv('test_sgca_regex.csv')

In [ ]:
temp[(temp['ls_cited_cases'].notnull()) & (temp['ls_cited_cases'] != '[]')]

In [ ]:
key_cases = (987,737,722)
contract_rankings, contract_ts = analyze_one_topic(edge_df, meta_df, '', 'contract', key_cases, with_prompts=False)

In [ ]:
format_rank_scores(contract_rankings).head(5).to_latex('Tables/contract_rankings.tex')

In [ ]:
plt.figure(figsize=(9,5))
styles = ('-', '--', ':')
labels = ('Zurich', 'Sembcorp', 'Sandar')
for i, case in enumerate(key_cases):
    contract_ts[case]['authority_score'].plot(label=labels[i], linestyle=styles[i], color='darkblue')
    plt.legend()

plt.ylabel('Authority Score')
plt.xlabel('Year')
plt.tight_layout()
plt.savefig('Images/zurich_ts.png')

In [ ]:
key_cases = (987,737,722)
_, global_ts = analyze_one_topic(edge_df, meta_df, '', '', key_cases, with_prompts=False)

In [ ]:
plt.figure(figsize=(10,6))
styles = ('-', '--', ':')
labels = ('Zurich', 'Sembcorp', 'Sandar')
for i, case in enumerate(key_cases):
    global_ts[case]['authority_score'].plot(label=labels[i], linestyle=styles[i], color='darkblue')
    plt.legend()

In [ ]:
find_cases_citing(722, edge_df[edge_df['citing_has_topic_contract']==1])

In [ ]:
find_cases_citing(987, edge_df[edge_df['citing_has_topic_contract']==1])

In [ ]:
contract_central_cases, contract_top_cases = analyze_one_topic(edge_df, 'contract', 'citing_has_topic_contract', with_prompts=False)

In [ ]:
contract_central_cases

In [ ]:
eda.convert_id_df_to_case_names(contract_central_cases)